In [1]:
import json
import os

In [2]:
jsonfiles = ["/root/data/gtsf_phase_I/2019-02-26/2019-02-26_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-02-27/2019-02-27_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-01/2019-03-01_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-04/2019-03-04_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-05/2019-03-05_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-06/2019-03-06_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-11/2019-03-11_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-13/2019-03-13_cogito_annotations.json",
             "/root/data/gtsf_phase_I/2019-03-14/2019-03-14_cogito_annotations.json"]

In [3]:
annotations = []
for jsonpath in jsonfiles:
    with open(jsonpath, "r") as f:
        jfile = json.load(f)
        annotations += jfile
print("Number of annotations: {}".format(len(annotations)))

Number of annotations: 485


In [4]:
for ann in annotations:
    local_path = os.path.join("/root/data/gtsf_phase_I/", 
                  "/".join(ann["Labeled Data"].split("/")[7:]))
    ann["local_path"] = local_path
    if not os.path.isfile(local_path):
        print("missing image!!")
    for body_part in ann["Label"].keys():
        new_body_part = "_".join(body_part.replace(":", "").split()).upper()
        ann["Label"][new_body_part] = ann["Label"].pop(body_part)

In [5]:
timestamps = []
for ann in annotations:
    timestamp = ann["local_path"].split("/")[-3]
    ann["timestamp"] = timestamp
    timestamps.append(ann["timestamp"])

In [6]:
# create pairs per timestamp
pairs = {}
for ann in annotations:
    timestamp = ann["timestamp"]
    side = os.path.basename(ann["local_path"]).split("_")[0]
    ann["side"] = side
    if timestamp not in pairs:
        pairs[timestamp] = {}
    pairs[timestamp][side] = ann

full_pairs = [k for (k, v)in pairs.items() if "left" in v and "right" in v]
print("Number of full pairs: {}".format(len(full_pairs)))

Number of full pairs: 240


Select a random pair and write input file

In [7]:
import numpy as np
from utils import load_keypoints

In [8]:
input_json = {}
random_timestanp = np.random.choice(full_pairs)

In [19]:
random_timestanp = '190306010013'

'190306010013'

In [9]:
mapping = {'ADIPOSE_FIN': 5,
 'ANAL_FIN': 1,
 'DORSAL_FIN': 4,
 'EYE': 7,
 'PECTORAL_FIN': 0,
 'PELVIC_FIN': 6,
 'TAIL_NOTCH': 3,
 'UPPER_LIP': 2}

In [10]:
world = {}
for ts in full_pairs:
    # load annotations
    left_ann = pairs[ts]["left"]
    right_ann = pairs[ts]["right"]
    
    left_keypoints = load_keypoints(left_ann, mapping)
    right_keypoints = load_keypoints(right_ann, mapping)

In [11]:
left_keypoints

array([[2016, 1150],
       [2630, 1618],
       [3299, 1434],
       [1259, 1405],
       [2793, 1311],
       [1139, 1405],
       [1480, 1553],
       [2179, 1650]])

In [12]:
right_keypoints

array([[1510, 1156],
       [2152, 1621],
       [2819, 1431],
       [ 754, 1411],
       [2293, 1310],
       [ 651, 1403],
       [ 974, 1557],
       [1673, 1653]])

In [13]:
left_crop = []
right_crop = []
for (k,v) in mapping.items():
    tmp = {}
    tmp["xFrame"] = int(left_keypoints[v, 1])
    tmp["yFrame"] = int(left_keypoints[v, 0])
    tmp["xCrop"] = 0
    tmp["yCrop"] = 0
    tmp["keypointType"] = k
    left_crop.append(tmp)
    
    tmp = {}
    tmp["xFrame"] = int(right_keypoints[v, 1])
    tmp["yFrame"] = int(right_keypoints[v, 0])
    tmp["xCrop"] = 0
    tmp["yCrop"] = 0
    tmp["keypointType"] = k
    right_crop.append(tmp)

In [14]:
left_crop

[{'keypointType': 'DORSAL_FIN',
  'xCrop': 0,
  'xFrame': 1311,
  'yCrop': 0,
  'yFrame': 2793},
 {'keypointType': 'ANAL_FIN',
  'xCrop': 0,
  'xFrame': 1618,
  'yCrop': 0,
  'yFrame': 2630},
 {'keypointType': 'TAIL_NOTCH',
  'xCrop': 0,
  'xFrame': 1405,
  'yCrop': 0,
  'yFrame': 1259},
 {'keypointType': 'EYE',
  'xCrop': 0,
  'xFrame': 1650,
  'yCrop': 0,
  'yFrame': 2179},
 {'keypointType': 'ADIPOSE_FIN',
  'xCrop': 0,
  'xFrame': 1405,
  'yCrop': 0,
  'yFrame': 1139},
 {'keypointType': 'UPPER_LIP',
  'xCrop': 0,
  'xFrame': 1434,
  'yCrop': 0,
  'yFrame': 3299},
 {'keypointType': 'PECTORAL_FIN',
  'xCrop': 0,
  'xFrame': 1150,
  'yCrop': 0,
  'yFrame': 2016},
 {'keypointType': 'PELVIC_FIN',
  'xCrop': 0,
  'xFrame': 1553,
  'yCrop': 0,
  'yFrame': 1480}]

In [15]:
right_crop

[{'keypointType': 'DORSAL_FIN',
  'xCrop': 0,
  'xFrame': 1310,
  'yCrop': 0,
  'yFrame': 2293},
 {'keypointType': 'ANAL_FIN',
  'xCrop': 0,
  'xFrame': 1621,
  'yCrop': 0,
  'yFrame': 2152},
 {'keypointType': 'TAIL_NOTCH',
  'xCrop': 0,
  'xFrame': 1411,
  'yCrop': 0,
  'yFrame': 754},
 {'keypointType': 'EYE',
  'xCrop': 0,
  'xFrame': 1653,
  'yCrop': 0,
  'yFrame': 1673},
 {'keypointType': 'ADIPOSE_FIN',
  'xCrop': 0,
  'xFrame': 1403,
  'yCrop': 0,
  'yFrame': 651},
 {'keypointType': 'UPPER_LIP',
  'xCrop': 0,
  'xFrame': 1431,
  'yCrop': 0,
  'yFrame': 2819},
 {'keypointType': 'PECTORAL_FIN',
  'xCrop': 0,
  'xFrame': 1156,
  'yCrop': 0,
  'yFrame': 1510},
 {'keypointType': 'PELVIC_FIN',
  'xCrop': 0,
  'xFrame': 1557,
  'yCrop': 0,
  'yFrame': 974}]

In [16]:
cameraParameters = {
      "baseline": 0.128096,
      "focalLength": 0.0084366,
      "pixelCountWidth": 4096,
      "focalLengthPixel": 2445.3913043478265,
      "imageSensorWidth": 0.01412,
      "pixelCountHeight": 3000,
      "imageSensorHeight": 0.01035
  }

In [17]:
input_json["leftCrop"] = left_crop
input_json["rightCrop"] = right_crop
input_json["cameraParameters"] = cameraParameters
input_json["annotationId"] = 10000
tmp["version"] = 1

In [18]:
with open("./sample_message.json", "w") as f:
    json.dump(input_json, f)